In [80]:
import pandas as pd
import numpy as np
import torch
from sklearn import preprocessing
from dgl.data import DGLDataset
import dgl
import time
import networkx as nx
import category_encoders as ce
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
import torch
import tqdm
import math

from typing import *
from sklearn.preprocessing import StandardScaler, Normalizer
import socket
import struct
import random
from sklearn.model_selection import train_test_split

In [81]:
file_name = "NF-UNSW-NB15-v2.parquet"
data = pd.read_parquet(file_name)

In [82]:
data.Label.value_counts()

Label
0    2295222
1      95053
Name: count, dtype: int64

In [83]:
data.rename(columns=lambda x: x.strip(), inplace=True)
data['IPV4_SRC_ADDR'] = data["IPV4_SRC_ADDR"].apply(str)
data['L4_SRC_PORT'] = data["L4_SRC_PORT"].apply(str)
data['IPV4_DST_ADDR'] = data["IPV4_DST_ADDR"].apply(str)
data['L4_DST_PORT'] = data["L4_DST_PORT"].apply(str)

In [84]:
data.drop(columns=["L4_SRC_PORT", "L4_DST_PORT"], inplace=True)

In [85]:
data.Attack.unique()

array(['Benign', 'Exploits', 'Generic', 'Fuzzers', 'Backdoor', 'DoS',
       'Reconnaissance', 'Shellcode', 'Worms', 'Analysis'], dtype=object)

In [86]:
# data = data.groupby(by='Attack').sample(frac=0.1, random_state=13)
# scale attack flows to 0.4
data_attack = data[data['Label'] == 1]
data_benign = data[data['Label'] == 0].sample(frac=0.02, random_state=13)
data = pd.concat([data_attack, data_benign], axis=0)
data.Label.value_counts()

Label
1    95053
0    45904
Name: count, dtype: int64

In [87]:
data.groupby(by="Attack").count()

,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,Label
Attack,,,,,,,,,,,,,,,,,,,,,
Analysis,2299,2299,2299,2299,2299,2299,2299,2299,2299,2299,...,2299,2299,2299,2299,2299,2299,2299,2299,2299,2299
Backdoor,2169,2169,2169,2169,2169,2169,2169,2169,2169,2169,...,2169,2169,2169,2169,2169,2169,2169,2169,2169,2169
Benign,45904,45904,45904,45904,45904,45904,45904,45904,45904,45904,...,45904,45904,45904,45904,45904,45904,45904,45904,45904,45904
DoS,5794,5794,5794,5794,5794,5794,5794,5794,5794,5794,...,5794,5794,5794,5794,5794,5794,5794,5794,5794,5794
Exploits,31551,31551,31551,31551,31551,31551,31551,31551,31551,31551,...,31551,31551,31551,31551,31551,31551,31551,31551,31551,31551
Fuzzers,22310,22310,22310,22310,22310,22310,22310,22310,22310,22310,...,22310,22310,22310,22310,22310,22310,22310,22310,22310,22310
Generic,16560,16560,16560,16560,16560,16560,16560,16560,16560,16560,...,16560,16560,16560,16560,16560,16560,16560,16560,16560,16560
Reconnaissance,12779,12779,12779,12779,12779,12779,12779,12779,12779,12779,...,12779,12779,12779,12779,12779,12779,12779,12779,12779,12779
Shellcode,1427,1427,1427,1427,1427,1427,1427,1427,1427,1427,...,1427,1427,1427,1427,1427,1427,1427,1427,1427,1427


In [88]:
X = data.drop(columns=["Attack", "Label"])
y = data[["Attack", "Label"]]

X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=13, stratify=y)

In [89]:
encoder = ce.TargetEncoder(cols=['TCP_FLAGS','L7_PROTO','PROTOCOL',
                                  'CLIENT_TCP_FLAGS','SERVER_TCP_FLAGS','ICMP_TYPE',
                                  'ICMP_IPV4_TYPE','DNS_QUERY_ID','DNS_QUERY_TYPE',
                                  'FTP_COMMAND_RET_CODE'])
encoder.fit(X_train, y_train.Label)

# Transform on training set
X_train = encoder.transform(X_train)

# Transform on testing set
X_test = encoder.transform(X_test)

In [90]:
X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
X_test.replace([np.inf, -np.inf], np.nan, inplace=True)
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

In [91]:
# (Modified)
scaler = Normalizer()
cols_to_norm = list(set(list(X_train.iloc[:, 2:].columns))) # Ignore first two as the represents IP addresses
scaler.fit(X_train[cols_to_norm])

# Transform on training set
X_train[cols_to_norm] = scaler.transform(X_train[cols_to_norm])
X_train['h'] = X_train.iloc[:, 2:].values.tolist()
X_train['id'] = X_train.index

# Transform on testing set
X_test[cols_to_norm] = scaler.transform(X_test[cols_to_norm])
X_test['h'] = X_test.iloc[:, 2:].values.tolist()
X_test['id'] = X_test.index

train = pd.concat([X_train, y_train], axis=1)
test = pd.concat([X_test, y_test], axis=1)

In [92]:
X_train.head()

,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,h,id
2303512,59.166.0.1,149.171.126.4,3.195953e-09,3.450790e-09,0.000014,2.265974e-07,0.000124,2.368973e-07,1.990142e-09,2.075070e-09,...,0.000142,0.000075,2.815574e-10,2.839759e-10,3.516510e-09,3.516932e-09,0.0,3.613093e-09,"[3.1959528356221716e-09, 3.450790320527692e-09...",2303512
1621997,175.45.176.0,149.171.126.10,5.996599e-08,6.474753e-08,0.000102,1.159547e-06,0.000072,7.730311e-07,9.627892e-08,9.149574e-08,...,0.001583,0.001583,9.418255e-08,9.430046e-08,6.598064e-08,6.598856e-08,0.0,6.779282e-08,"[5.996599021772413e-08, 6.474753203418179e-08,...",1621997
1353664,59.166.0.8,149.171.126.7,3.117918e-08,3.366533e-08,0.000052,6.029037e-07,0.000114,6.029037e-07,1.941549e-08,2.024404e-08,...,0.000509,0.000509,9.562671e-09,1.052265e-08,3.430648e-08,3.431060e-08,0.0,3.524873e-08,"[3.117918247798922e-08, 3.366533444980148e-08,...",1353664
1853118,175.45.176.0,149.171.126.19,5.141805e-07,4.985595e-07,0.000125,1.488095e-06,0.000000,0.000000e+00,5.812618e-07,5.812618e-07,...,0.000000,0.000000,5.571449e-07,5.571500e-07,5.080544e-07,5.081154e-07,0.0,5.220084e-07,"[5.141805153463133e-07, 4.985594668314381e-07,...",1853118
1223489,175.45.176.0,149.171.126.19,1.331376e-07,2.111734e-07,0.000102,2.145372e-06,0.000072,1.716298e-06,2.137602e-07,2.031405e-07,...,0.003515,0.003515,2.123919e-07,2.124644e-07,1.464914e-07,1.465090e-07,0.0,1.505149e-07,"[1.3313759613512936e-07, 2.1117340154866449e-0...",1223489


In [93]:
lab_enc = preprocessing.LabelEncoder()
lab_enc.fit(data["Attack"])

# Transform on training set
train["Attack"] = lab_enc.transform(train["Attack"])

# Transform on testing set
test["Attack"] = lab_enc.transform(test["Attack"])

In [94]:
# Training graph (Modified)

train['id'] = train.index

train_g = nx.from_pandas_edgelist(train, "IPV4_SRC_ADDR", "IPV4_DST_ADDR",
           ["h", "Label", "Attack", "id"], create_using=nx.MultiGraph())
train_g = train_g.to_directed()
train_g = dgl.from_networkx(train_g, edge_attrs=['h', 'Attack', 'Label', "id"])
nfeat_weight = torch.ones([train_g.number_of_nodes(),
train_g.edata['h'].shape[1]])
train_g.ndata['h'] = nfeat_weight

test['id'] = test.index

# Testing graph
test_g = nx.from_pandas_edgelist(test, "IPV4_SRC_ADDR", "IPV4_DST_ADDR",
            ["h", "Label", "Attack", "id"], create_using=nx.MultiGraph())
# print(test_g)
test_g = test_g.to_directed()
# print(test_g)
test_g = dgl.from_networkx(test_g, edge_attrs=['h', 'Attack', 'Label', "id"])
nfeat_weight = torch.ones([test_g.number_of_nodes(),
test_g.edata['h'].shape[1]])
test_g.ndata['h'] = nfeat_weight

In [95]:
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
import tqdm
import gc

class SAGELayer(nn.Module):
    def __init__(self, ndim_in, edims, ndim_out, activation):
      super(SAGELayer, self).__init__()
      self.W_apply = nn.Linear(ndim_in + edims , ndim_out)
      self.activation = F.relu
      self.W_edge = nn.Linear(128 * 2, 256)
      self.reset_parameters()

    def reset_parameters(self):
      gain = nn.init.calculate_gain('relu')
      nn.init.xavier_uniform_(self.W_apply.weight, gain=gain)

    def message_func(self, edges):
      return {'m':  edges.data['h']}

    def forward(self, g_dgl, nfeats, efeats):
      with g_dgl.local_scope():
        g = g_dgl
        g.ndata['h'] = nfeats
        g.edata['h'] = efeats
        g.update_all(self.message_func, fn.mean('m', 'h_neigh'))
        g.ndata['h'] = F.relu(self.W_apply(torch.cat([g.ndata['h'], g.ndata['h_neigh']], 2)))

        # Compute edge embeddings
        u, v = g.edges()
        edge = self.W_edge(torch.cat((g.srcdata['h'][u], g.dstdata['h'][v]), 2))
        return g.ndata['h'], edge

In [96]:
class SAGE(nn.Module):
    def __init__(self, ndim_in, ndim_out, edim,  activation):
      super(SAGE, self).__init__()
      self.layers = nn.ModuleList()
      self.layers.append(SAGELayer(ndim_in, edim, 128, F.relu))

    def forward(self, g, nfeats, efeats, corrupt=False):
      if corrupt:
        e_perm = torch.randperm(g.number_of_edges())
        #n_perm = torch.randperm(g.number_of_nodes())
        efeats = efeats[e_perm]
        #nfeats = nfeats[n_perm]
      for i, layer in enumerate(self.layers):
        #nfeats = layer(g, nfeats, efeats)
        nfeats, e_feats = layer(g, nfeats, efeats)
      #return nfeats.sum(1)
      return nfeats.sum(1), e_feats.sum(1)

In [97]:
class Discriminator(nn.Module):
    def __init__(self, n_hidden):
      super(Discriminator, self).__init__()
      self.weight = nn.Parameter(torch.Tensor(n_hidden, n_hidden))
      self.reset_parameters()

    def uniform(self, size, tensor):
      bound = 1.0 / math.sqrt(size)
      if tensor is not None:
        tensor.data.uniform_(-bound, bound)

    def reset_parameters(self):
      size = self.weight.size(0)
      self.uniform(size, self.weight)

    def forward(self, features, summary):
      features = torch.matmul(features, torch.matmul(self.weight, summary))
      return features

In [98]:
class DGI(nn.Module):
    def __init__(self, ndim_in, ndim_out, edim, activation):
      super(DGI, self).__init__()
      self.encoder = SAGE(ndim_in, ndim_out, edim,  F.relu)
      #self.discriminator = Discriminator(128)
      self.discriminator = Discriminator(256)
      self.loss = nn.BCEWithLogitsLoss()

    def forward(self, g, n_features, e_features):
      positive = self.encoder(g, n_features, e_features, corrupt=False)
      negative = self.encoder(g, n_features, e_features, corrupt=True)
      self.loss = nn.BCEWithLogitsLoss()

    def forward(self, g, n_features, e_features):
      positive = self.encoder(g, n_features, e_features, corrupt=False)
      negative = self.encoder(g, n_features, e_features, corrupt=True)

      positive = positive[1]
      negative = negative[1]

      summary = torch.sigmoid(positive.mean(dim=0))

      positive = self.discriminator(positive, summary)
      negative = self.discriminator(negative, summary)

      l1 = self.loss(positive, torch.ones_like(positive))
      l2 = self.loss(negative, torch.zeros_like(negative))

      return l1 + l2

In [99]:
ndim_in = train_g.ndata['h'].shape[1]
hidden_features = 128
ndim_out = 128
num_layers = 1
edim = train_g.edata['h'].shape[1]
learning_rate = 1e-3
epochs = 4000

In [100]:
dgi = DGI(ndim_in,
    ndim_out,
    edim,
    F.relu)

dgi = dgi.to('cuda')

dgi_optimizer = torch.optim.Adam(dgi.parameters(),
                lr=1e-3,
                weight_decay=0.)

In [101]:
# Format node and edge features for E-GraphSAGE
train_g.ndata['h'] = torch.reshape(train_g.ndata['h'],
                                   (train_g.ndata['h'].shape[0], 1,
                                    train_g.ndata['h'].shape[1]))

train_g.edata['h'] = torch.reshape(train_g.edata['h'],
                                   (train_g.edata['h'].shape[0], 1,
                                    train_g.edata['h'].shape[1]))

In [102]:
# Convert to GPU
train_g = train_g.to('cuda')

In [103]:
cnt_wait = 0
best = 1e9
best_t = 0
dur = []
node_features = train_g.ndata['h'] 
edge_features = train_g.edata['h']

for epoch in range(epochs):
    dgi.train()
    if epoch >= 3:
        t0 = time.time()

    dgi_optimizer.zero_grad()
    loss = dgi(train_g, node_features, edge_features)
    loss.backward()
    dgi_optimizer.step()

    if loss < best:
        best = loss
        best_t = epoch
        cnt_wait = 0
        torch.save(dgi.state_dict(), 'best_dgi_UNSW_v2.pkl')
    else:
        cnt_wait += 1

  # if cnt_wait == patience:
  #     print('Early stopping!')
  #     break

    if epoch >= 3:
        dur.append(time.time() - t0)

    if epoch % 50 == 0:

        print("Epoch {:05d} | Time(s) {:.4f} | Loss {:.4f} | "
            "ETputs(KTEPS) {:.2f}".format(epoch, np.mean(dur),
              loss.item(),
              train_g.num_edges() / np.mean(dur) / 1000))

/home/kienho/D/DesktopFiles/Hoc/DACN/paper_codes/Anomal-E/.conda/lib/python3.9/site-packages/numpy/_core/fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/kienho/D/DesktopFiles/Hoc/DACN/paper_codes/Anomal-E/.conda/lib/python3.9/site-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Epoch 00000 | Time(s) nan | Loss 1.4068 | ETputs(KTEPS) nan
Epoch 00050 | Time(s) 0.0984 | Loss 1.3912 | ETputs(KTEPS) 2005.47
Epoch 00100 | Time(s) 0.0996 | Loss 1.3270 | ETputs(KTEPS) 1980.80
Epoch 00150 | Time(s) 0.1001 | Loss 1.0624 | ETputs(KTEPS) 1971.90
Epoch 00200 | Time(s) 0.1001 | Loss 0.2465 | ETputs(KTEPS) 1970.89
Epoch 00250 | Time(s) 0.1002 | Loss 0.0595 | ETputs(KTEPS) 1968.54
Epoch 00300 | Time(s) 0.1003 | Loss 0.0322 | ETputs(KTEPS) 1967.59
Epoch 00350 | Time(s) 0.1002 | Loss 0.0219 | ETputs(KTEPS) 1968.73
Epoch 00400 | Time(s) 0.1002 | Loss 0.0153 | ETputs(KTEPS) 1969.60
Epoch 00450 | Time(s) 0.1002 | Loss 0.0112 | ETputs(KTEPS) 1970.01
Epoch 00500 | Time(s) 0.1002 | Loss 0.0091 | ETputs(KTEPS) 1969.92
Epoch 00550 | Time(s) 0.1001 | Loss 0.0078 | ETputs(KTEPS) 1970.50
Epoch 00600 | Time(s) 0.1001 | Loss 0.0056 | ETputs(KTEPS) 1970.79
Epoch 00650 | Time(s) 0.1001 | Loss 0.0061 | ETputs(KTEPS) 1971.26
Epoch 00700 | Time(s) 0.1001 | Loss 0.0041 | ETputs(KTEPS) 1971.41
Ep

In [104]:
dgi.load_state_dict(torch.load('best_dgi_UNSW_v2.pkl'))

<All keys matched successfully>

In [106]:
training_emb = dgi.encoder(train_g, train_g.ndata['h'], train_g.edata['h'])[1]
training_emb = training_emb.detach().cpu().numpy()

In [107]:
test_g.ndata['h'] = torch.reshape(test_g.ndata['h'],
                                   (test_g.ndata['h'].shape[0], 1,
                                    test_g.ndata['h'].shape[1]))



test_g.edata['h'] = torch.reshape(test_g.edata['h'],
                                   (test_g.edata['h'].shape[0], 1,
                                    test_g.edata['h'].shape[1]))

In [108]:
# Convert to GPU
test_g = test_g.to('cuda')

In [109]:
testing_emb = dgi.encoder(test_g, test_g.ndata['h'], test_g.edata['h'])[1]
testing_emb = testing_emb.detach().cpu().numpy()

In [110]:
# Multimodal (Fusion) Learning

df_train = pd.DataFrame(training_emb,)
# map the id to the original data
df_train['id'] = train_g.edata['id'].detach().cpu().numpy()


df_raw_train = pd.DataFrame(X_train.drop(columns=["IPV4_SRC_ADDR", "IPV4_DST_ADDR", "h"]))
df_fuse_train = pd.merge(df_train, df_raw_train, on='id', how='left')
df_fuse_train = df_fuse_train.drop(columns=["id"])
df_fuse_train["Attacks"] = train_g.edata['Attack'].detach().cpu().numpy()
df_fuse_train["Label"] = train_g.edata['Label'].detach().cpu().numpy()

df_test = pd.DataFrame(testing_emb,)
# map the id to the original data
df_test['id'] = test_g.edata['id'].detach().cpu().numpy()

df_raw_test = pd.DataFrame(X_test.drop(columns=["IPV4_SRC_ADDR", "IPV4_DST_ADDR", "h"]))
df_raw_test = pd.merge(df_test, df_raw_test, on='id', how='left')
df_fuse_test = df_raw_test.drop(columns=["id"])
df_fuse_test["Attacks"] = test_g.edata['Attack'].detach().cpu().numpy()
df_fuse_test["Label"] = test_g.edata['Label'].detach().cpu().numpy()

# Embeddings CBLOF  Embeddings

In [111]:
import torch
import dgl
import numpy as np
import pandas as pd
import torch.optim as optim
import time
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from sklearn.ensemble import IsolationForest
from pyod.models.cblof import CBLOF
import gc

from tqdm import tqdm
import itertools

In [112]:
benign_fuse_train_samples = df_fuse_train[df_fuse_train.Label == 0].drop(columns=["Label", "Attacks"])
normal_fuse_train_samples = df_fuse_train.drop(columns=["Label", "Attacks"])

fuse_train_labels = df_fuse_train["Label"]
fuse_test_labels = df_fuse_test["Label"]

fuse_test_samples = df_fuse_test.drop(columns=["Label", "Attacks"])

In [33]:
fuse_test_samples

,0,1,2,3,4,5,6,7,8,9,...,NUM_PKTS_512_TO_1024_BYTES,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE
0,-0.011583,0.019045,-0.010781,-0.001299,-0.100072,0.026116,-0.018159,0.024632,-0.029538,0.013363,...,0.000000e+00,8.370473e-08,0.000108,0.000043,2.356220e-12,2.356220e-12,1.233329e-10,1.233338e-10,0.000000,1.335599e-10
1,-0.011583,0.019045,-0.010781,-0.001299,-0.100072,0.026116,-0.018159,0.024632,-0.029538,0.013363,...,0.000000e+00,0.000000e+00,0.000000,0.000000,3.092057e-08,3.094159e-08,2.158972e-08,3.577436e-10,0.000033,2.425808e-08
2,-0.011583,0.019045,-0.010781,-0.001299,-0.100072,0.026116,-0.018159,0.024632,-0.029538,0.013363,...,0.000000e+00,0.000000e+00,0.000000,0.000000,1.155203e-08,1.155988e-08,8.368929e-09,8.368988e-09,0.000000,9.062899e-09
3,-0.011583,0.019045,-0.010781,-0.001299,-0.100072,0.026116,-0.018159,0.024632,-0.029538,0.013363,...,0.000000e+00,0.000000e+00,0.000000,0.000000,3.381315e-08,3.383613e-08,2.360940e-08,3.912099e-10,0.000036,2.652738e-08
4,-0.011583,0.019045,-0.010781,-0.001299,-0.100072,0.026116,-0.018159,0.024632,-0.029538,0.013363,...,9.504689e-09,8.554220e-08,0.000089,0.000023,2.206288e-12,2.206288e-12,6.535423e-11,6.535469e-11,0.000000,7.077354e-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143413,0.016703,0.058107,0.000909,-0.003440,-0.063689,0.158615,0.000451,0.061308,-0.000357,-0.006514,...,0.000000e+00,0.000000e+00,0.000000,0.000000,1.326019e-08,1.326921e-08,9.606417e-09,9.606485e-09,0.000000,1.040300e-08
143414,0.016703,0.058107,0.000909,-0.003440,-0.063689,0.158615,0.000451,0.061308,-0.000357,-0.006514,...,0.000000e+00,0.000000e+00,0.000000,0.000000,1.308516e-08,1.309405e-08,9.479610e-09,9.479677e-09,0.000000,1.026568e-08
143415,0.016703,0.058107,0.000909,-0.003440,-0.063689,0.158615,0.000451,0.061308,-0.000357,-0.006514,...,0.000000e+00,0.000000e+00,0.000000,0.000000,1.326024e-08,1.326926e-08,9.606453e-09,9.606521e-09,0.000000,1.040304e-08
143416,0.016703,0.058107,0.000909,-0.003440,-0.063689,0.158615,0.000451,0.061308,-0.000357,-0.006514,...,0.000000e+00,0.000000e+00,0.000000,0.000000,1.326020e-08,1.326921e-08,9.606419e-09,9.606487e-09,0.000000,1.040300e-08


In [34]:
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]

In [ ]:
n_est = [5,6,7,9,10] # cant be lower than 5 or 4
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    try:
        clf_if = CBLOF(n_clusters=n_est, contamination=con)
        clf_if.fit(benign_fuse_train_samples)
    except Exception as e:
        print(n_est)
        continue
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

 47%|████▋     | 14/30 [02:41<02:37,  9.86s/it]Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7fa0da177850>>
Traceback (most recent call last):
  File "/home/kienho/.local/lib/python3.9/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 
 53%|█████▎    | 16/30 [03:03<02:26, 10.45s/it]

In [ ]:
n_est = [5,6,7,9,10]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    try:
        clf_if = CBLOF(n_clusters=n_est, contamination=con)
        clf_if.fit(normal_fuse_train_samples)
    except Exception as e:
        print(n_est)
        continue
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 30/30 [05:13<00:00, 10.44s/it]

{'n_estimators': 5, 'con': 0.2}
0.7246850923456903
              precision    recall  f1-score   support

           0     0.9713    0.8682    0.9168    374702
           1     0.4050    0.7773    0.5325     43236

    accuracy                         0.8588    417938
   macro avg     0.6881    0.8228    0.7247    417938
weighted avg     0.9127    0.8588    0.8771    417938



In [ ]:
# HBOS  Embeddings+Raw (Multimodal)

In [ ]:
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]

In [ ]:
from pyod.models.hbos import HBOS

n_est = [5,10,15,20,25,30]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = HBOS(n_bins=n_est, contamination=con)
    clf_if.fit(benign_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 36/36 [05:45<00:00,  9.60s/it]

{'n_estimators': 25, 'con': 0.05}
0.8471039149382792
              precision    recall  f1-score   support

           0     1.0000    0.9177    0.9571    374702
           1     0.5837    1.0000    0.7371     43236

    accuracy                         0.9262    417938
   macro avg     0.7918    0.9588    0.8471    417938
weighted avg     0.9569    0.9262    0.9343    417938



In [ ]:
n_est = [5,10,15,20,25,30]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = HBOS(n_bins=n_est, contamination=con)
    clf_if.fit(normal_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 36/36 [06:05<00:00, 10.16s/it]

{'n_estimators': 25, 'con': 0.1}
0.8471039149382792
              precision    recall  f1-score   support

           0     1.0000    0.9177    0.9571    374702
           1     0.5837    1.0000    0.7371     43236

    accuracy                         0.9262    417938
   macro avg     0.7918    0.9588    0.8471    417938
weighted avg     0.9569    0.9262    0.9343    417938



In [ ]:
##  PCA  Emb+Raw (Multimodal/Fusion) Learning

In [ ]:
from pyod.models.pca import PCA
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(benign_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 36/36 [05:07<00:00,  8.55s/it]

{'n_estimators': 5, 'con': 0.1}
0.8207887532486517
              precision    recall  f1-score   support

           0     1.0000    0.8987    0.9466    374702
           1     0.5325    1.0000    0.6949     43236

    accuracy                         0.9092    417938
   macro avg     0.7662    0.9493    0.8208    417938
weighted avg     0.9516    0.9092    0.9206    417938



In [ ]:
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(normal_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 36/36 [05:49<00:00,  9.69s/it]

{'n_estimators': 15, 'con': 0.2}
0.8085873162698853
              precision    recall  f1-score   support

           0     1.0000    0.8893    0.9414    374702
           1     0.5103    1.0000    0.6758     43236

    accuracy                         0.9007    417938
   macro avg     0.7552    0.9446    0.8086    417938
weighted avg     0.9493    0.9007    0.9139    417938



In [ ]:
##  IF  Emb+Raw

In [ ]:
for i in range(len(benign_fuse_train_samples.columns)):
    benign_fuse_train_samples.rename(columns={benign_fuse_train_samples.columns[i]: f"feature {i}"}, inplace=True)

for i in range(len(normal_fuse_train_samples.columns)):
    normal_fuse_train_samples.rename(columns={normal_fuse_train_samples.columns[i]: f"feature {i}"}, inplace=True)

for i in range(len(fuse_test_samples.columns)):
    fuse_test_samples.rename(columns={fuse_test_samples.columns[i]: f"feature {i}"}, inplace=True)

In [ ]:
from sklearn.ensemble import IsolationForest
n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(benign_fuse_train_samples.to_numpy())
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

  0%|          | 0/24 [00:00<?, ?it/s]/home/kienho/D/DesktopFiles/Hoc/DACN/paper_codes/Anomal-E/.conda/lib/python3.9/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but IsolationForest was fitted without feature names
  warnings.warn(
  4%|▍         | 1/24 [00:02<00:54,  2.36s/it]/home/kienho/D/DesktopFiles/Hoc/DACN/paper_codes/Anomal-E/.conda/lib/python3.9/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but IsolationForest was fitted without feature names
  warnings.warn(
  8%|▊         | 2/24 [00:04<00:51,  2.34s/it]/home/kienho/D/DesktopFiles/Hoc/DACN/paper_codes/Anomal-E/.conda/lib/python3.9/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but IsolationForest was fitted without feature names
  warnings.warn(
 12%|█▎        | 3/24 [00:07<00:48,  2.33s/it]/home/kienho/D/DesktopFiles/Hoc/DACN/paper_codes/Anomal-E/.conda/lib/python3.9/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but IsolationForest wa

{'n_estimators': 50, 'con': 0.1}
0.820368662501618
              precision    recall  f1-score   support

           0     1.0000    0.8984    0.9465    374702
           1     0.5317    1.0000    0.6943     43236

    accuracy                         0.9089    417938
   macro avg     0.7659    0.9492    0.8204    417938
weighted avg     0.9516    0.9089    0.9204    417938



In [ ]:
from sklearn.ensemble import IsolationForest

n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(normal_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 24/24 [01:52<00:00,  4.70s/it]

{'n_estimators': 150, 'con': 0.2}
0.802806141381684
              precision    recall  f1-score   support

           0     0.9991    0.8861    0.9392    374702
           1     0.5015    0.9928    0.6664     43236

    accuracy                         0.8972    417938
   macro avg     0.7503    0.9395    0.8028    417938
weighted avg     0.9476    0.8972    0.9110    417938



## Supervised attack classification (Fusion)

We now train a supervised classifier on the fused features to predict multi-class attack labels:
- Features: embeddings + raw numeric features from `df_fuse_train`/`df_fuse_test` (without `Label`, `Attacks`).
- Target: `Attacks` (encoded integer classes from earlier `LabelEncoder`).
- Model: HistGradientBoostingClassifier (fast, strong on tabular data). Class imbalance handled via per-sample weights.
- Metrics: macro F1, per-class report, and confusion matrix.

In [113]:
# Prepare supervised train/test for attack classification
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, f1_score, confusion_matrix
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.utils import class_weight
import pickle

# Build train features/targets from already prepared fused DataFrames
X_sup_train = df_fuse_train.drop(columns=["Label", "Attacks"]).copy()
y_sup_train = df_fuse_train["Attacks"].copy()

X_sup_test = df_fuse_test.drop(columns=["Label", "Attacks"]).copy()
y_sup_test = df_fuse_test["Attacks"].copy()

# Compute sample weights to mitigate class imbalance on training set
classes = np.unique(y_sup_train)
class_w = class_weight.compute_class_weight(
    class_weight="balanced", classes=classes, y=y_sup_train
)
class_to_w = {c: w for c, w in zip(classes, class_w)}
sample_weight = y_sup_train.map(class_to_w).values

# Feature names to all str
X_sup_train.columns = X_sup_train.columns.map(str)
X_sup_test.columns = X_sup_test.columns.map(str)

## Classification Methods (Fused Features)

Now we'll compare multiple classification algorithms on the fused features:

In [114]:
# Random Forest with Grid Search
from sklearn.ensemble import RandomForestClassifier
import pickle

print("="*60)
print("RANDOM FOREST CLASSIFIER (Fused Features)")
print("="*60)

try:
    n_estimators_grid = [100, 200, 300]
    max_depth_grid = [10, 20, 30]
    params = list(itertools.product(n_estimators_grid, max_depth_grid))

    score = -1
    best_params = None
    best_model = None

    print(f"Testing {len(params)} configurations...")

    for n_est, depth in tqdm(params):
        try:
            rf_clf = RandomForestClassifier(
                n_estimators=n_est,
                max_depth=depth,
                class_weight='balanced',
                random_state=13,
                n_jobs=-1
            )
            
            rf_clf.fit(X_sup_train, y_sup_train)
            y_pred_rf = rf_clf.predict(X_sup_test)
            rf_f1 = f1_score(y_sup_test, y_pred_rf, average='macro')
            
            if rf_f1 > score:
                score = rf_f1
                best_params = {
                    'n_estimators': n_est,
                    'max_depth': depth
                }
                best_model = rf_clf
                # Save best model
                with open('best_rf_classifier_fused.pkl', 'wb') as f:
                    pickle.dump(rf_clf, f)
        except Exception as e:
            print(f"\nError with params (n={n_est}, d={depth}): {str(e)}")
            continue

    if best_model is not None:
        print("\nBest Parameters:", best_params)
        print(f"Best Macro F1: {score:.4f}")
        print("Model saved to: best_rf_classifier_fused.pkl\n")
        print(classification_report(y_sup_test, best_model.predict(X_sup_test), digits=4))
    else:
        print("\nAll configurations failed.")
        
except Exception as e:
    print(f"Unexpected error in Random Forest: {str(e)}")

RANDOM FOREST CLASSIFIER (Fused Features)
Testing 9 configurations...


100%|██████████| 9/9 [02:25<00:00, 16.12s/it]



Best Parameters: {'n_estimators': 200, 'max_depth': 20}
Best Macro F1: 0.5047
Model saved to: best_rf_classifier_fused.pkl

              precision    recall  f1-score   support

           0     0.3207    0.0551    0.0940      1380
           1     0.1572    0.1966    0.1747      1302
           2     0.9999    0.9919    0.9959     27542
           3     0.3168    0.1343    0.1887      3476
           4     0.8013    0.8095    0.8054     18932
           5     0.8225    0.7663    0.7934     13386
           6     0.9524    0.5997    0.7360      9936
           7     0.7150    0.7930    0.7520      7668
           8     0.0656    0.5724    0.1176       856
           9     0.4268    0.3571    0.3889        98

    accuracy                         0.7835     84576
   macro avg     0.5578    0.5276    0.5047     84576
weighted avg     0.8337    0.7835    0.7984     84576



In [115]:
# XGBoost with Grid Search
try:
    from xgboost import XGBClassifier
    import pickle
    
    print("="*60)
    print("XGBOOST CLASSIFIER (Fused Features)")
    print("="*60)
    
    n_estimators_grid = [100, 200, 300]
    max_depth_grid = [6, 8, 10]
    learning_rate_grid = [0.01, 0.05, 0.1]
    
    params = list(itertools.product(n_estimators_grid, max_depth_grid, learning_rate_grid))
    
    score = -1
    best_params = None
    best_model = None
    
    print(f"Testing {len(params)} configurations...")
    
    for n_est, depth, lr in tqdm(params):
        try:
            xgb_clf = XGBClassifier(
                n_estimators=n_est,
                max_depth=depth,
                learning_rate=lr,
                random_state=13,
                tree_method='hist', 
                device="cuda"
            )
            
            xgb_clf.fit(X_sup_train, y_sup_train, sample_weight=sample_weight)
            y_pred_xgb = xgb_clf.predict(X_sup_test)
            xgb_f1 = f1_score(y_sup_test, y_pred_xgb, average='macro')
            
            if xgb_f1 > score:
                score = xgb_f1
                best_params = {
                    'n_estimators': n_est,
                    'max_depth': depth,
                    'learning_rate': lr
                }
                best_model = xgb_clf
                # Save best model
                xgb_clf.save_model('best_xgb_classifier_fused.json')
        except Exception as e:
            print(f"\nError with params (n={n_est}, d={depth}, lr={lr}): {str(e)}")
            continue
    
    if best_model is not None:
        print("\nBest Parameters:", best_params)
        print(f"Best Macro F1: {score:.4f}")
        print("Model saved to: best_xgb_classifier_fused.json\n")
        print(classification_report(y_sup_test, best_model.predict(X_sup_test), digits=4))
    else:
        print("\nAll configurations failed. XGBoost might not support your GPU.")
    
except ImportError:
    print("XGBoost not installed. Install with: pip install xgboost")
except Exception as e:
    print(f"Unexpected error: {str(e)}")

XGBOOST CLASSIFIER (Fused Features)
Testing 27 configurations...


  0%|          | 0/27 [00:00<?, ?it/s]/home/kienho/D/DesktopFiles/Hoc/DACN/paper_codes/Anomal-E/.conda/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [13:58:50] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)
100%|██████████| 27/27 [25:12<00:00, 56.02s/it] 



Best Parameters: {'n_estimators': 300, 'max_depth': 10, 'learning_rate': 0.1}
Best Macro F1: 0.6648
Model saved to: best_xgb_classifier_fused.json

              precision    recall  f1-score   support

           0     0.1573    0.3072    0.2081      1380
           1     0.1482    0.5753    0.2356      1302
           2     0.9869    0.9950    0.9909     27542
           3     0.4138    0.3452    0.3764      3476
           4     0.9039    0.8070    0.8527     18932
           5     0.9400    0.8501    0.8928     13386
           6     0.9639    0.8427    0.8992      9936
           7     0.8679    0.7969    0.8309      7668
           8     0.5650    0.8680    0.6845       856
           9     0.6509    0.7041    0.6765        98

    accuracy                         0.8481     84576
   macro avg     0.6598    0.7092    0.6648     84576
weighted avg     0.8927    0.8481    0.8661     84576



In [116]:
# CatBoost with Grid Search (Expanded, no l2_leaf_reg/border_count, fixed best score logic)
try:
    from catboost import CatBoostClassifier
    
    print("Starting CatBoost Grid Search (Expanded)...")
    
    # Expanded hyperparameter grid
    iterations_grid = [200, 300, 500]
    depth_grid = [4, 8, 10]
    learning_rate_grid = [0.01, 0.05, 0.1]
    
    params = list(itertools.product(iterations_grid, depth_grid, learning_rate_grid))
    print(f"Testing {len(params)} configurations...")
    
    best_score = -1
    best_params = None
    best_model = None
    best_model_path = None
    
    for iterations, depth, lr in tqdm(params):
        try:
            cat_clf = CatBoostClassifier(
                iterations=iterations,
                depth=depth,
                learning_rate=lr,
                auto_class_weights='Balanced',
                random_seed=13,
                verbose=False,
                task_type='GPU'  # Use 'CPU' if GPU not available
            )
            
            cat_clf.fit(X_sup_train, y_sup_train)
            y_pred_cat = cat_clf.predict(X_sup_test)
            cat_f1 = f1_score(y_sup_test, y_pred_cat, average='macro')
            
            if cat_f1 > best_score:
                best_score = cat_f1
                best_params = {
                    'iterations': iterations,
                    'depth': depth,
                    'learning_rate': lr
                }
                best_model = cat_clf
                # Save the best model with unique name for fused features
                best_model_path = "best_catboost_classifier_fused.cbm"
                best_model.save_model(best_model_path)
        except Exception as e:
            print(f"\nError with params (it={iterations}, d={depth}, lr={lr}): {str(e)}")
            continue
    
    print("\n" + "="*60)
    print("BEST CATBOOST HYPERPARAMETERS (Fused Features):")
    print(best_params)
    print(f"Best Macro F1: {best_score:.4f}")
    print("="*60)
    if best_model_path:
        print(f"\nBest CatBoost model saved to: {best_model_path}")
    
    # Final evaluation
    y_pred_best = best_model.predict(X_sup_test)
    print("\n" + "="*60)
    print("FINAL CLASSIFICATION REPORT (Best CatBoost - Fused):")
    print("="*60)
    print(classification_report(y_sup_test, y_pred_best, digits=4))
    
except ImportError:
    print("CatBoost not installed. Install with: pip install catboost")

Starting CatBoost Grid Search (Expanded)...
Testing 27 configurations...


100%|██████████| 27/27 [12:46<00:00, 28.39s/it]


BEST CATBOOST HYPERPARAMETERS (Fused Features):
{'iterations': 500, 'depth': 10, 'learning_rate': 0.1}
Best Macro F1: 0.6008

Best CatBoost model saved to: best_catboost_classifier_fused.cbm

FINAL CLASSIFICATION REPORT (Best CatBoost - Fused):
              precision    recall  f1-score   support

           0     0.1760    0.3899    0.2426      1380
           1     0.1521    0.5661    0.2398      1302
           2     0.9999    0.9945    0.9972     27542
           3     0.3115    0.3501    0.3297      3476
           4     0.9102    0.7263    0.8079     18932
           5     0.9003    0.7905    0.8418     13386
           6     0.9386    0.7966    0.8618      9936
           7     0.8405    0.8129    0.8264      7668
           8     0.3444    0.9428    0.5045       856
           9     0.2322    0.7653    0.3563        98

    accuracy                         0.8187     84576
   macro avg     0.5806    0.7135    0.6008     84576
weighted avg     0.8801    0.8187    0.8417     84

In [117]:
# Extra Trees Classifier
from sklearn.ensemble import ExtraTreesClassifier
import pickle

print("="*60)
print("EXTRA TREES CLASSIFIER (Fused Features)")
print("="*60)

try:
    n_estimators_grid = [100, 200, 300]
    max_depth_grid = [10, 20, 30]

    params = list(itertools.product(n_estimators_grid, max_depth_grid))

    score = -1
    best_params = None
    best_model = None

    print(f"Testing {len(params)} configurations...")

    for n_est, depth in tqdm(params):
        try:
            et_clf = ExtraTreesClassifier(
                n_estimators=n_est,
                max_depth=depth,
                class_weight='balanced',
                random_state=13,
                n_jobs=-1
            )
            
            et_clf.fit(X_sup_train, y_sup_train)
            y_pred_et = et_clf.predict(X_sup_test)
            et_f1 = f1_score(y_sup_test, y_pred_et, average='macro')
            
            if et_f1 > score:
                score = et_f1
                best_params = {
                    'n_estimators': n_est,
                    'max_depth': depth
                }
                best_model = et_clf
                # Save best model
                with open('best_et_classifier_fused.pkl', 'wb') as f:
                    pickle.dump(et_clf, f)
        except Exception as e:
            print(f"\nError with params (n={n_est}, d={depth}): {str(e)}")
            continue

    if best_model is not None:
        print("\nBest Parameters:", best_params)
        print(f"Best Macro F1: {score:.4f}")
        print("Model saved to: best_et_classifier_fused.pkl\n")
        print(classification_report(y_sup_test, best_model.predict(X_sup_test), digits=4))
    else:
        print("\nAll configurations failed.")
        
except Exception as e:
    print(f"Unexpected error in Extra Trees: {str(e)}")

EXTRA TREES CLASSIFIER (Fused Features)
Testing 9 configurations...


100%|██████████| 9/9 [01:38<00:00, 10.97s/it]



Best Parameters: {'n_estimators': 200, 'max_depth': 30}
Best Macro F1: 0.4646
Model saved to: best_et_classifier_fused.pkl

              precision    recall  f1-score   support

           0     0.2551    0.0457    0.0774      1380
           1     0.1271    0.1382    0.1325      1302
           2     0.9909    0.9902    0.9906     27542
           3     0.2846    0.0964    0.1440      3476
           4     0.7625    0.8391    0.7989     18932
           5     0.8322    0.7370    0.7817     13386
           6     0.9069    0.4902    0.6364      9936
           7     0.7298    0.7578    0.7436      7668
           8     0.0480    0.4544    0.0868       856
           9     0.2771    0.2347    0.2541        98

    accuracy                         0.7649     84576
   macro avg     0.5214    0.4784    0.4646     84576
weighted avg     0.8164    0.7649    0.7777     84576



## Raw Features Classification (Comparison Baseline)

Now we'll train classifiers on **raw features only** (without graph embeddings) to compare the benefit of multimodal fusion:

In [118]:
# Prepare raw features (without embeddings)
# Use only the original numeric features from X_train/X_test

X_raw_train = X_train.drop(columns=["IPV4_SRC_ADDR", "IPV4_DST_ADDR", "h", "id"]).copy()
y_raw_train = y_train["Attack"].copy()

X_raw_test = X_test.drop(columns=["IPV4_SRC_ADDR", "IPV4_DST_ADDR", "h", "id"]).copy()
y_raw_test = y_test["Attack"].copy()

# Encode labels
y_raw_train_encoded = lab_enc.transform(y_train["Attack"])
y_raw_test_encoded = lab_enc.transform(y_test["Attack"])

# Compute sample weights
classes_raw = np.unique(y_raw_train_encoded)
class_w_raw = class_weight.compute_class_weight(
    class_weight="balanced", classes=classes_raw, y=y_raw_train_encoded
)
class_to_w_raw = {c: w for c, w in zip(classes_raw, class_w_raw)}
sample_weight_raw = pd.Series(y_raw_train_encoded).map(class_to_w_raw).values

# Feature names to str
X_raw_train.columns = X_raw_train.columns.map(str)
X_raw_test.columns = X_raw_test.columns.map(str)

print(f"Raw feature shape - Train: {X_raw_train.shape}, Test: {X_raw_test.shape}")
print(f"Number of classes: {len(np.unique(y_raw_train_encoded))}")

Raw feature shape - Train: (98669, 39), Test: (42288, 39)
Number of classes: 10


In [119]:
# Raw Features - Random Forest
from sklearn.ensemble import RandomForestClassifier
import pickle

print("="*60)
print("RANDOM FOREST (Raw Features Only)")

n_estimators_grid = [100, 200, 300]
max_depth_grid = [10, 20, 30]
params = list(itertools.product(n_estimators_grid, max_depth_grid))

score = -1
best_params = None
best_model = None

print(f"Testing {len(params)} configurations...")

for n_est, depth in tqdm(params):
    rf_clf = RandomForestClassifier(
        n_estimators=n_est,
        max_depth=depth,
        class_weight='balanced',
        random_state=13,
        n_jobs=-1
    )
    
    rf_clf.fit(X_raw_train, y_raw_train_encoded)
    y_pred = rf_clf.predict(X_raw_test)
    f1 = f1_score(y_raw_test_encoded, y_pred, average='macro')
    
    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est, 'max_depth': depth}
        best_model = rf_clf
        # Save best model
        with open('best_rf_classifier_raw.pkl', 'wb') as f:
            pickle.dump(rf_clf, f)

print("\nBest Parameters:", best_params)
print(f"Best Macro F1: {score:.4f}")
print("Model saved to: best_rf_classifier_raw.pkl\n")
print(classification_report(y_raw_test_encoded, best_model.predict(X_raw_test), digits=4))

RANDOM FOREST (Raw Features Only)
Testing 9 configurations...


100%|██████████| 9/9 [00:46<00:00,  5.14s/it]



Best Parameters: {'n_estimators': 300, 'max_depth': 30}
Best Macro F1: 0.6044
Model saved to: best_rf_classifier_raw.pkl

              precision    recall  f1-score   support

           0     0.1585    0.8420    0.2668       690
           1     0.1820    0.1828    0.1824       651
           2     0.9985    0.9932    0.9959     13771
           3     0.5497    0.2768    0.3682      1738
           4     0.9020    0.7907    0.8427      9466
           5     0.9219    0.7950    0.8538      6693
           6     0.9584    0.8247    0.8865      4968
           7     0.8106    0.8125    0.8115      3834
           8     0.3601    0.8388    0.5039       428
           9     0.2048    0.8776    0.3320        49

    accuracy                         0.8343     42288
   macro avg     0.6046    0.7234    0.6044     42288
weighted avg     0.8909    0.8343    0.8536     42288



In [120]:
# Raw Features - XGBoost
try:
    from xgboost import XGBClassifier
    
    print("="*60)
    print("XGBOOST (Raw Features Only)")
    print("="*60)
    
    n_estimators_grid = [100, 200, 300]
    max_depth_grid = [6, 8, 10]
    learning_rate_grid = [0.01, 0.05, 0.1]
    
    params = list(itertools.product(n_estimators_grid, max_depth_grid, learning_rate_grid))
    
    score = -1
    best_params = None
    best_model = None
    
    print(f"Testing {len(params)} configurations...")
    
    for n_est, depth, lr in tqdm(params):
        xgb_clf = XGBClassifier(
            n_estimators=n_est,
            max_depth=depth,
            learning_rate=lr,
            random_state=13,
            tree_method='hist',
            device='cuda'
        )
        
        xgb_clf.fit(X_raw_train, y_raw_train_encoded, sample_weight=sample_weight_raw)
        y_pred = xgb_clf.predict(X_raw_test)
        f1 = f1_score(y_raw_test_encoded, y_pred, average='macro')
        
        if f1 > score:
            score = f1
            best_params = {'n_estimators': n_est, 'max_depth': depth, 'learning_rate': lr}
            best_model = xgb_clf
            # Save best model
            xgb_clf.save_model('best_xgb_classifier_raw.json')
    
    print("\nBest Parameters:", best_params)
    print(f"Best Macro F1: {score:.4f}")
    print("Model saved to: best_xgb_classifier_raw.json\n")
    print(classification_report(y_raw_test_encoded, best_model.predict(X_raw_test), digits=4))
    
except ImportError:
    print("XGBoost not installed.")

XGBOOST (Raw Features Only)
Testing 27 configurations...


100%|██████████| 27/27 [04:43<00:00, 10.49s/it]



Best Parameters: {'n_estimators': 300, 'max_depth': 10, 'learning_rate': 0.1}
Best Macro F1: 0.6716
Model saved to: best_xgb_classifier_raw.json

              precision    recall  f1-score   support

           0     0.1538    0.6797    0.2509       690
           1     0.1434    0.2780    0.1892       651
           2     0.9993    0.9946    0.9969     13771
           3     0.4683    0.3487    0.3997      1738
           4     0.9211    0.7907    0.8510      9466
           5     0.9456    0.8624    0.9021      6693
           6     0.9707    0.8476    0.9050      4968
           7     0.8546    0.8112    0.8323      3834
           8     0.5704    0.9276    0.7064       428
           9     0.5676    0.8571    0.6829        49

    accuracy                         0.8506     42288
   macro avg     0.6595    0.7398    0.6716     42288
weighted avg     0.9032    0.8506    0.8711     42288



In [121]:
# Raw Features - CatBoost (Expanded grid, no l2_leaf_reg/border_count)
try:
    from catboost import CatBoostClassifier
    
    print("="*60)
    print("CATBOOST (Raw Features Only, Expanded)")
    print("="*60)
    
    iterations_grid = [200, 300, 500]
    depth_grid = [4, 8, 10]
    learning_rate_grid = [0.01, 0.05, 0.1]
    
    params = list(itertools.product(iterations_grid, depth_grid, learning_rate_grid))
    
    best_score = -1
    best_params = None
    best_model = None
    best_model_path = None
    
    print(f"Testing {len(params)} configurations...")
    
    for iterations, depth, lr in tqdm(params):
        try:
            cat_clf = CatBoostClassifier(
                iterations=iterations,
                depth=depth,
                learning_rate=lr,
                auto_class_weights='Balanced',
                random_seed=13,
                verbose=False,
                task_type='GPU'
            )
            
            cat_clf.fit(X_raw_train, y_raw_train_encoded)
            y_pred = cat_clf.predict(X_raw_test)
            f1 = f1_score(y_raw_test_encoded, y_pred, average='macro')
            
            if f1 > best_score:
                best_score = f1
                best_params = {'iterations': iterations, 'depth': depth, 'learning_rate': lr}
                best_model = cat_clf
                # Save the best model with unique name for raw features
                best_model_path = "best_catboost_classifier_raw.cbm"
                best_model.save_model(best_model_path)
        except Exception as e:
            print(f"\nError with params (it={iterations}, d={depth}, lr={lr}): {str(e)}")
            continue
    
    print("\nBest Parameters:", best_params)
    print(f"Best Macro F1: {best_score:.4f}\n")
    if best_model_path:
        print(f"Best CatBoost model saved to: {best_model_path}")
    print(classification_report(y_raw_test_encoded, best_model.predict(X_raw_test), digits=4))
    
except ImportError:
    print("CatBoost not installed.")

CATBOOST (Raw Features Only, Expanded)
Testing 27 configurations...


100%|██████████| 27/27 [03:53<00:00,  8.64s/it]


Best Parameters: {'iterations': 500, 'depth': 10, 'learning_rate': 0.1}
Best Macro F1: 0.6138

Best CatBoost model saved to: best_catboost_classifier_raw.cbm
              precision    recall  f1-score   support

           0     0.1578    0.7884    0.2629       690
           1     0.1566    0.2704    0.1983       651
           2     0.9998    0.9935    0.9966     13771
           3     0.3842    0.3435    0.3627      1738
           4     0.9193    0.7242    0.8101      9466
           5     0.9206    0.8061    0.8596      6693
           6     0.9548    0.8082    0.8754      4968
           7     0.8465    0.8159    0.8309      3834
           8     0.3760    0.9533    0.5393       428
           9     0.2571    0.9184    0.4018        49

    accuracy                         0.8240     42288
   macro avg     0.5973    0.7422    0.6138     42288
weighted avg     0.8909    0.8240    0.8483     42288



In [122]:
# Raw Features - Extra Trees
from sklearn.ensemble import ExtraTreesClassifier
import pickle

print("="*60)
print("EXTRA TREES (Raw Features Only)")
print("="*60)

n_estimators_grid = [100, 200, 300]
max_depth_grid = [10, 20, 30]

params = list(itertools.product(n_estimators_grid, max_depth_grid))

score = -1
best_params = None
best_model = None

print(f"Testing {len(params)} configurations...")

for n_est, depth in tqdm(params):
    et_clf = ExtraTreesClassifier(
        n_estimators=n_est,
        max_depth=depth,
        class_weight='balanced',
        random_state=13,
        n_jobs=-1
    )
    
    et_clf.fit(X_raw_train, y_raw_train_encoded)
    y_pred = et_clf.predict(X_raw_test)
    f1 = f1_score(y_raw_test_encoded, y_pred, average='macro')
    
    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est, 'max_depth': depth}
        best_model = et_clf
        # Save best model
        with open('best_et_classifier_raw.pkl', 'wb') as f:
            pickle.dump(et_clf, f)

print("\nBest Parameters:", best_params)
print(f"Best Macro F1: {score:.4f}")
print("Model saved to: best_et_classifier_raw.pkl\n")
print(classification_report(y_raw_test_encoded, best_model.predict(X_raw_test), digits=4))

EXTRA TREES (Raw Features Only)
Testing 9 configurations...


100%|██████████| 9/9 [00:28<00:00,  3.14s/it]



Best Parameters: {'n_estimators': 200, 'max_depth': 30}
Best Macro F1: 0.6525
Model saved to: best_et_classifier_raw.pkl

              precision    recall  f1-score   support

           0     0.1584    0.8420    0.2666       690
           1     0.1747    0.1951    0.1843       651
           2     0.9983    0.9946    0.9964     13771
           3     0.5066    0.3084    0.3834      1738
           4     0.9130    0.8008    0.8532      9466
           5     0.9304    0.8345    0.8798      6693
           6     0.9568    0.8428    0.8962      4968
           7     0.8541    0.7833    0.8171      3834
           8     0.4659    0.9089    0.6160       428
           9     0.5000    0.8571    0.6316        49

    accuracy                         0.8448     42288
   macro avg     0.6458    0.7367    0.6525     42288
weighted avg     0.8980    0.8448    0.8640     42288



## Embeddings Only Classification (Graph Features)

Now we'll train classifiers on **embeddings only** (graph features without raw features) to isolate the value of graph-based learning:

In [123]:
# Prepare embeddings-only features (graph features without raw data)
# Extract only the embedding columns (first 256 dimensions from graph encoder)

# From the fused dataframes, extract only embedding columns
# df_fuse_train has: [0-255] = embeddings, [256+] = raw features
num_embedding_dims = 256  # Based on the DGI encoder output dimension

X_emb_train = df_fuse_train.iloc[:, :num_embedding_dims].copy()
y_emb_train = df_fuse_train["Attacks"].copy()

X_emb_test = df_fuse_test.iloc[:, :num_embedding_dims].copy()
y_emb_test = df_fuse_test["Attacks"].copy()

# Compute sample weights
classes_emb = np.unique(y_emb_train)
class_w_emb = class_weight.compute_class_weight(
    class_weight="balanced", classes=classes_emb, y=y_emb_train
)
class_to_w_emb = {c: w for c, w in zip(classes_emb, class_w_emb)}
sample_weight_emb = y_emb_train.map(class_to_w_emb).values

# Feature names to str
X_emb_train.columns = X_emb_train.columns.map(str)
X_emb_test.columns = X_emb_test.columns.map(str)

print(f"Embeddings shape - Train: {X_emb_train.shape}, Test: {X_emb_test.shape}")
print(f"Number of classes: {len(np.unique(y_emb_train))}")
print(f"Using only graph embeddings (no raw features)")

Embeddings shape - Train: (197338, 256), Test: (84576, 256)
Number of classes: 10
Using only graph embeddings (no raw features)


In [124]:
# Embeddings Only - Random Forest
from sklearn.ensemble import RandomForestClassifier
import pickle

print("="*60)
print("RANDOM FOREST (Embeddings Only)")
print("="*60)

n_estimators_grid = [100, 200, 300]
max_depth_grid = [10, 20, 30]
params = list(itertools.product(n_estimators_grid, max_depth_grid))

score = -1
best_params = None
best_model = None

print(f"Testing {len(params)} configurations...")

for n_est, depth in tqdm(params):
    rf_clf = RandomForestClassifier(
        n_estimators=n_est,
        max_depth=depth,
        class_weight='balanced',
        random_state=13,
        n_jobs=-1
    )
    
    rf_clf.fit(X_emb_train, y_emb_train)
    y_pred = rf_clf.predict(X_emb_test)
    f1 = f1_score(y_emb_test, y_pred, average='macro')
    
    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est, 'max_depth': depth}
        best_model = rf_clf
        # Save best model
        with open('best_rf_classifier_embeddings.pkl', 'wb') as f:
            pickle.dump(rf_clf, f)

print("\nBest Parameters:", best_params)
print(f"Best Macro F1: {score:.4f}")
print("Model saved to: best_rf_classifier_embeddings.pkl\n")
print(classification_report(y_emb_test, best_model.predict(X_emb_test), digits=4))

RANDOM FOREST (Embeddings Only)
Testing 9 configurations...


  0%|          | 0/9 [00:00<?, ?it/s]

100%|██████████| 9/9 [01:03<00:00,  7.08s/it]
/home/kienho/D/DesktopFiles/Hoc/DACN/paper_codes/Anomal-E/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Best Parameters: {'n_estimators': 100, 'max_depth': 20}
Best Macro F1: 0.1944
Model saved to: best_rf_classifier_embeddings.pkl

              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000      1380
           1     0.0663    0.4539    0.1157      1302
           2     1.0000    0.9858    0.9929     27542
           3     0.0532    0.0178    0.0267      3476
           4     0.4365    0.0383    0.0704     18932
           5     0.2781    0.3266    0.3004     13386
           6     0.3324    0.4207    0.3714      9936
           7     0.1557    0.0198    0.0352      7668
           8     0.0154    0.1016    0.0267       856
           9     0.0021    0.2347    0.0042        98

    accuracy                         0.4415     84576
   macro avg     0.2340    0.2599    0.1944     84576
weighted avg     0.5239    0.4415    0.4366     84576



/home/kienho/D/DesktopFiles/Hoc/DACN/paper_codes/Anomal-E/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/D/DesktopFiles/Hoc/DACN/paper_codes/Anomal-E/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [125]:
# Embeddings Only - XGBoost
try:
    from xgboost import XGBClassifier
    
    print("="*60)
    print("XGBOOST (Embeddings Only)")
    print("="*60)
    
    n_estimators_grid = [100, 200, 300]
    max_depth_grid = [6, 8, 10]
    learning_rate_grid = [0.01, 0.05, 0.1]
    
    params = list(itertools.product(n_estimators_grid, max_depth_grid, learning_rate_grid))
    
    score = -1
    best_params = None
    best_model = None
    
    print(f"Testing {len(params)} configurations...")
    
    for n_est, depth, lr in tqdm(params):
        xgb_clf = XGBClassifier(
            n_estimators=n_est,
            max_depth=depth,
            learning_rate=lr,
            random_state=13,
            tree_method='hist',
            device='gpu'
        )
        
        xgb_clf.fit(X_emb_train, y_emb_train, sample_weight=sample_weight_emb)
        y_pred = xgb_clf.predict(X_emb_test)
        f1 = f1_score(y_emb_test, y_pred, average='macro')
        
        if f1 > score:
            score = f1
            best_params = {'n_estimators': n_est, 'max_depth': depth, 'learning_rate': lr}
            best_model = xgb_clf
            # Save best model
            xgb_clf.save_model('best_xgb_classifier_embeddings.json')
    
    print("\nBest Parameters:", best_params)
    print(f"Best Macro F1: {score:.4f}")
    print("Model saved to: best_xgb_classifier_embeddings.json\n")
    print(classification_report(y_emb_test, best_model.predict(X_emb_test), digits=4))
    
except ImportError:
    print("XGBoost not installed.")

XGBOOST (Embeddings Only)
Testing 27 configurations...


100%|██████████| 27/27 [13:05<00:00, 29.10s/it]



Best Parameters: {'n_estimators': 200, 'max_depth': 6, 'learning_rate': 0.05}
Best Macro F1: 0.2050
Model saved to: best_xgb_classifier_embeddings.json

              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000      1380
           1     0.0715    0.1866    0.1034      1302
           2     0.9660    0.9859    0.9758     27542
           3     0.0967    0.1536    0.1187      3476
           4     0.3745    0.1354    0.1989     18932
           5     0.2825    0.2542    0.2676     13386
           6     0.2818    0.3091    0.2948      9936
           7     0.1721    0.0346    0.0576      7668
           8     0.0168    0.1589    0.0303       856
           9     0.0014    0.1020    0.0029        98

    accuracy                         0.4420     84576
   macro avg     0.2263    0.2320    0.2050     84576
weighted avg     0.4971    0.4420    0.4513     84576



In [126]:
# Embeddings Only - CatBoost (Expanded grid, no l2_leaf_reg/border_count)
try:
    from catboost import CatBoostClassifier
    
    print("="*60)
    print("CATBOOST (Embeddings Only, Expanded)")
    print("="*60)
    
    iterations_grid = [200, 300, 500]
    depth_grid = [4, 8, 10]
    learning_rate_grid = [0.01, 0.05, 0.1]
    
    params = list(itertools.product(iterations_grid, depth_grid, learning_rate_grid))
    
    best_score = -1
    best_params = None
    best_model = None
    best_model_path = None
    
    print(f"Testing {len(params)} configurations...")
    
    for iterations, depth, lr in tqdm(params):
        try:
            cat_clf = CatBoostClassifier(
                iterations=iterations,
                depth=depth,
                learning_rate=lr,
                auto_class_weights='Balanced',
                random_seed=13,
                verbose=False,
                task_type='GPU'
            )
            
            cat_clf.fit(X_emb_train, y_emb_train)
            y_pred = cat_clf.predict(X_emb_test)
            f1 = f1_score(y_emb_test, y_pred, average='macro')
            
            if f1 > best_score:
                best_score = f1
                best_params = {'iterations': iterations, 'depth': depth, 'learning_rate': lr}
                best_model = cat_clf
                # Save the best model with unique name for embeddings
                best_model_path = "best_catboost_classifier_embeddings.cbm"
                best_model.save_model(best_model_path)
        except Exception as e:
            print(f"\nError with params (it={iterations}, d={depth}, lr={lr}): {str(e)}")
            continue
    
    print("\nBest Parameters:", best_params)
    print(f"Best Macro F1: {best_score:.4f}\n")
    if best_model_path:
        print(f"Best CatBoost model saved to: {best_model_path}")
    print(classification_report(y_emb_test, best_model.predict(X_emb_test), digits=4))
    
except ImportError:
    print("CatBoost not installed.")

CATBOOST (Embeddings Only, Expanded)
Testing 27 configurations...


100%|██████████| 27/27 [09:30<00:00, 21.13s/it]


Best Parameters: {'iterations': 500, 'depth': 8, 'learning_rate': 0.05}
Best Macro F1: 0.1950

Best CatBoost model saved to: best_catboost_classifier_embeddings.cbm
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000      1380
           1     0.0711    0.2473    0.1105      1302
           2     1.0000    0.9858    0.9929     27542
           3     0.0881    0.1657    0.1151      3476
           4     0.3932    0.0632    0.1089     18932
           5     0.3046    0.0728    0.1176     13386
           6     0.3672    0.4114    0.3881      9936
           7     0.1487    0.0561    0.0814      7668
           8     0.0156    0.3166    0.0296       856
           9     0.0029    0.2551    0.0057        98

    accuracy                         0.4142     84576
   macro avg     0.2391    0.2574    0.1950     84576
weighted avg     0.5234    0.4142    0.4260     84576




/home/kienho/D/DesktopFiles/Hoc/DACN/paper_codes/Anomal-E/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/D/DesktopFiles/Hoc/DACN/paper_codes/Anomal-E/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/D/DesktopFiles/Hoc/DACN/paper_codes/Anomal-E/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division

In [127]:
# Embeddings Only - Extra Trees
from sklearn.ensemble import ExtraTreesClassifier
import pickle

print("="*60)
print("EXTRA TREES (Embeddings Only)")
print("="*60)

n_estimators_grid = [100, 200, 300]
max_depth_grid = [10, 20, 30]

params = list(itertools.product(n_estimators_grid, max_depth_grid))

score = -1
best_params = None
best_model = None

print(f"Testing {len(params)} configurations...")

for n_est, depth in tqdm(params):
    et_clf = ExtraTreesClassifier(
        n_estimators=n_est,
        max_depth=depth,
        class_weight='balanced',
        random_state=13,
        n_jobs=-1
    )
    
    et_clf.fit(X_emb_train, y_emb_train)
    y_pred = et_clf.predict(X_emb_test)
    f1 = f1_score(y_emb_test, y_pred, average='macro')
    
    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est, 'max_depth': depth}
        best_model = et_clf
        # Save best model
        with open('best_et_classifier_embeddings.pkl', 'wb') as f:
            pickle.dump(et_clf, f)

print("\nBest Parameters:", best_params)
print(f"Best Macro F1: {score:.4f}")
print("Model saved to: best_et_classifier_embeddings.pkl\n")
print(classification_report(y_emb_test, best_model.predict(X_emb_test), digits=4))

EXTRA TREES (Embeddings Only)
Testing 9 configurations...


100%|██████████| 9/9 [00:40<00:00,  4.45s/it]



Best Parameters: {'n_estimators': 200, 'max_depth': 20}
Best Macro F1: 0.1906
Model saved to: best_et_classifier_embeddings.pkl

              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000      1380
           1     0.0521    0.2281    0.0848      1302
           2     1.0000    0.9858    0.9929     27542
           3     0.0532    0.0089    0.0153      3476
           4     0.3719    0.1086    0.1681     18932
           5     0.2681    0.2784    0.2731     13386
           6     0.2842    0.3867    0.3276      9936
           7     0.1557    0.0099    0.0186      7668
           8     0.0136    0.0421    0.0205       856
           9     0.0027    0.4082    0.0053        98

    accuracy                         0.4405     84576
   macro avg     0.2201    0.2457    0.1906     84576
weighted avg     0.5020    0.4405    0.4465     84576



/home/kienho/D/DesktopFiles/Hoc/DACN/paper_codes/Anomal-E/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/D/DesktopFiles/Hoc/DACN/paper_codes/Anomal-E/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/D/DesktopFiles/Hoc/DACN/paper_codes/Anomal-E/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division`

## Summary Comparison Table

Create a comparison table of all methods (Fused vs Raw features):

In [49]:
# Create a summary comparison table
# NOTE: After running all cells above, manually collect the F1 scores and create a comparison

print("="*95)
print("PERFORMANCE COMPARISON: FUSED vs EMBEDDINGS vs RAW FEATURES")
print("="*95)
print("\nInstructions:")
print("1. Run all cells above to get F1 scores for each method")
print("2. Record the best Macro F1 score for each method")
print("3. Compare three approaches:")
print("   - Fused Features: Embeddings + Raw (Multimodal)")
print("   - Embeddings Only: Graph features only")
print("   - Raw Features: Traditional features only")
print("\nExpected format:")
print("-" * 95)
print(f"{'Method':<20} {'Fused (Emb+Raw)':<25} {'Embeddings Only':<25} {'Raw Only':<25}")
print("-" * 95)
print(f"{'Random Forest':<20} {'[INSERT SCORE]':<25} {'[INSERT SCORE]':<25} {'[INSERT SCORE]':<25}")
print(f"{'XGBoost':<20} {'[INSERT SCORE]':<25} {'[INSERT SCORE]':<25} {'[INSERT SCORE]':<25}")
print(f"{'CatBoost':<20} {'[INSERT SCORE]':<25} {'[INSERT SCORE]':<25} {'[INSERT SCORE]':<25}")
print(f"{'Extra Trees':<20} {'[INSERT SCORE]':<25} {'[INSERT SCORE]':<25} {'[INSERT SCORE]':<25}")
print("-" * 95)
print("\nAnalysis:")
print("- Fused features should show best performance (multimodal learning)")
print("- Embeddings capture structural/graph patterns")
print("- Raw features provide traditional statistical information")
print("- Compare to understand the contribution of each modality")

PERFORMANCE COMPARISON: FUSED vs EMBEDDINGS vs RAW FEATURES

Instructions:
1. Run all cells above to get F1 scores for each method
2. Record the best Macro F1 score for each method
3. Compare three approaches:
   - Fused Features: Embeddings + Raw (Multimodal)
   - Embeddings Only: Graph features only
   - Raw Features: Traditional features only

Expected format:
-----------------------------------------------------------------------------------------------
Method               Fused (Emb+Raw)           Embeddings Only           Raw Only                 
-----------------------------------------------------------------------------------------------
Random Forest        [INSERT SCORE]            [INSERT SCORE]            [INSERT SCORE]           
XGBoost              [INSERT SCORE]            [INSERT SCORE]            [INSERT SCORE]           
CatBoost             [INSERT SCORE]            [INSERT SCORE]            [INSERT SCORE]           
Extra Trees          [INSERT SCORE]           

## Key Insights & Interpretation

After running all experiments, analyze the results to answer:

1. **Which approach performs best overall?**
   - Fused features (multimodal) should ideally outperform single modalities
   - Compare the magnitude of improvements

2. **What is the value of graph embeddings?**
   - Compare Embeddings-only vs Raw-only to see if graph structure helps
   - If embeddings alone beat raw features, graph learning is beneficial

3. **Is multimodal fusion effective?**
   - Compare Fused vs (Embeddings + Raw separately)
   - Synergy should provide additional gains beyond individual modalities

4. **Which classifier is most suitable?**
   - Identify the best performing algorithm for each feature type
   - Consider computational cost vs accuracy trade-offs

5. **Feature contribution analysis:**
   - If Fused ≈ Embeddings > Raw: Graph structure dominates
   - If Fused ≈ Raw > Embeddings: Traditional features dominate
   - If Fused > Both: True synergy from multimodal learning